# Машинное обучение, DS-поток
## Задание 1.12


**Правила:**

* Выполненную работу нужно отправить телеграм-боту `@miptstats_ds21_bot`.
* Дедлайны см. в боте. После дедлайна работы не принимаются кроме случаев наличия уважительной причины.
* Прислать нужно ноутбук в формате `ipynb`.
* Решения, размещенные на каких-либо интернет-ресурсах не принимаются. Публикация решения может быть приравнена к предоставлении возможности списать.
* Для выполнения задания используйте этот ноутбук в качестве основы, ничего не удаляя из него.
* Никакой код из данного задания при проверке запускаться не будет.
* За задание можно получить до **20 баллов**:
  * часть 1 &mdash; 5 баллов, 
  * часть 2 &mdash; 15 баллов.


In [1]:
!pip install pycocotools

In [1]:
import numpy as np
from random import choice
import matplotlib.pyplot as plt
import json
from collections import Counter
from pathlib import Path
from tqdm.notebook import tqdm

%env CUDA_VISIBLE_DEVICES=0
import torch, torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torchvision import models
from torchvision.datasets import coco
from torchvision import transforms

from sklearn.model_selection import train_test_split

from nltk.tokenize import TweetTokenizer

env: CUDA_VISIBLE_DEVICES=0


# Image Captioning

### План работы.
**Часть 1.** В качестве энкодера, переводящего изображения в векторы, взять предобученную сеть inception v3. Эту часть можно пропустить с потерей баллов, взяв готовые векторы, ссылка на которые есть во второй части.

**Часть 2.** В качестве декодера, переводящего векторы в текст описания, взять LSTM; обучить декодер на векторизованных изображениях датасета [MSCOCO](http://cocodataset.org/#download).

Задача image captioning заключается в генерации текстовых описаний к изображениям. В большинстве случаев используются архитектуры из двух частей &mdash; энкодера, переводящего изображения в векторы, и декодера, генерирующего текст по этим векторам. Поскольку обучать энкодер и декодер вместе &mdash; вычислительно затратная операция, мы решили разделить их и разбить домашнее задание на две части.

## Часть 1. Векторизация изображений

![](https://docs.google.com/uc?export=download&id=18Dp1dng87NbnhlBHnFVoEaiKlNZEXybh)


В этой части вам предстоит выполнить первый этап задачи image captioning &mdash; обучения энкодера для перевода изображений в векторы. 

**Замечание.** Выполнение этого задания требует больших вычислительных ресурсов. Рекомендуется использовать Google Colab или ноутбук с GPU. Кроме того, вы можете пропустить эту часть с потерей баллов и начать сразу с генерации текстов по векторам.

### Данные

Для решения задачи используется датасет COCO (Common Objects in Context), содержащий большое количество изображений для image captioning.

Загрузим данные. Понадобится около 18 гигабайт на диске.

In [2]:
# If you work with colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Загрузим данные с http://cocodataset.org/#download
root = "./drive/My\ Drive/Colab\ Notebooks" # "./"

# !curl http://images.cocodataset.org/zips/train2017.zip > $root/train2017.zip
# !curl http://images.cocodataset.org/zips/val2017.zip > $root/val2017.zip
# !curl http://images.cocodataset.org/annotations/annotations_trainval2017.zip > $root/annotations_trainval2017.zip

!unzip $root/annotations_trainval2017.zip
!unzip $root/train2017.zip > log
!unzip $root/val2017.zip > log

Archive:  ./drive/My Drive/Colab Notebooks/annotations_trainval2017.zip
replace annotations/instances_train2017.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: 
error:  invalid response [{ENTER}]
replace annotations/instances_train2017.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: annotations/instances_train2017.json  
  inflating: annotations/instances_val2017.json  
  inflating: annotations/captions_train2017.json  
  inflating: annotations/captions_val2017.json  
  inflating: annotations/person_keypoints_train2017.json  
  inflating: annotations/person_keypoints_val2017.json  
replace train2017/000000147328.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
y
replace val2017/000000212226.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
y


In [4]:
!rm ./log

Чтобы упростить вам задачу, мы не будем обучать энкодер, а возьмём предобученные веса для сети. Предлагается  использовать сеть `ResNet`, но вы можете выбрать любую сеть для изображений на свой вкус.

Важно помнить, что перед передачей в сеть изображения нужно предобработать. При этом, у каждой сети свои требования к размеру входному изображению и способу нормирования каналов. 

In [5]:
# предобработка для Inception-v3
preprocess = transforms.Compose((
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
))

Инициализируем `DataLoader` для генерации батчей. Значение `batch_size` выберите в зависимости от имеющихся вычислительных ресурсов.

In [6]:
coco_train = coco.CocoCaptions(
    "./train2017/", "./annotations/captions_train2017.json", transform=preprocess,
    target_transform=(lambda x: x[:5]) # one picture has 6 captions
)

data_loader = torch.utils.data.DataLoader(
    dataset=coco_train, 
    batch_size=32, 
    shuffle=False, 
    num_workers=4
)

loading annotations into memory...
Done (t=1.40s)
creating index...
index created!


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Модифицируем предобученную сеть для генерации векторов по изображениям. Как было сказано выше, вы можете выбрать любую архитектуру, но размер выхода должен быть равным 2048. 

В `torchvision.models` содержатся сети, предобученные на задаче классификации. Поэтому для их использования надо удалить из них последний слой.

In [7]:
class EncoderCNN(nn.Module):
    def __init__(self):
        super(EncoderCNN, self).__init__()
        # загрузим предобученную сеть
        net = models.resnet50(pretrained=True)

        for param in net.parameters():
            # отключим подсчёт градиента для параметра
            param.requires_grad_(False)
        
        # выберем все слои, кроме последнего. Воспользуйтесь методом children() 
        # у класса предобученной сети
        modules = [child for child in net.children()][:-1]
        self.resnet = nn.Sequential(*modules)

    def forward(self, images):
        features = self.resnet(images)
        features = features.view(features.size(0), -1)
        return features

In [8]:
model = EncoderCNN()
# установим режим evaluation. Это отключит все dropout-ы в сети
model = nn.DataParallel(model.train(False).cuda())

### Векторизация изображений

Получим векторы представления изображений с помощью модели.

**Замечание.** Требует большого объёма RAM. Если памяти не хватает, то следует каждые $K$ итераций сохранять векторы на диск и очищать память.


In [10]:
vectors, captions = [], []

for img_batch, capt_batch in tqdm(data_loader):
    capt_batch = list(zip(*capt_batch))
    img_batch = Variable(img_batch, volatile=True)
    # получите векторы изображений из модели
    vec_batch = model(img_batch).cpu().numpy()
    
    captions.extend(capt_batch)
    vectors.extend([vec for vec in vec_batch])
    

  0%|          | 0/3697 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  """


Переведите тексты описаний в нижний регистр и токенизируйте их.

In [18]:
tokenizer = TweetTokenizer()
# токенизируем описания текстов
captions_tokenized = []

for text in tqdm(captions):
    text_tokenized = []
    for sentence in text:
        tokens = tokenizer.tokenize(sentence)
        text_tokenized.append(' '.join(tokens).lower())
    captions_tokenized.append(text_tokenized)

captions_tokenized

  0%|          | 0/118287 [00:00<?, ?it/s]

[['closeup of bins of food that include broccoli and bread .',
  'a meal is presented in brightly colored plastic trays .',
  'there are containers filled with different kinds of foods',
  'colorful dishes holding meat , vegetables , fruit , and bread .',
  'a bunch of trays that have different food .'],
 ['a giraffe eating food from the top of the tree .',
  'a giraffe standing up nearby a tree',
  'a giraffe mother with its baby in the forest .',
  'two giraffes standing in a tree filled area .',
  'a giraffe standing next to a forest filled with trees .'],
 ['a flower vase is sitting on a porch stand .',
  'white vase with different colored flowers sitting inside of it .',
  'a white vase with many flowers on a stage',
  'a white vase filled with different colored flowers .',
  'a vase with red and white flowers outside on a sunny day .'],
 ['a zebra grazing on lush green grass in a field .',
  'zebra reaching its head down to ground where grass is .',
  'the zebra is eating grass i

In [19]:
print("Исходный текст:\n%s\n\n" % '\n'.join(captions[0]))
print("Токенизированный текст:\n%s\n\n"% '\n'.join(["[%s]" % '],['.join(token) for token in captions_tokenized[0]]))

Исходный текст:
Closeup of bins of food that include broccoli and bread.
A meal is presented in brightly colored plastic trays.
there are containers filled with different kinds of foods
Colorful dishes holding meat, vegetables, fruit, and bread.
A bunch of trays that have different food.


Токенизированный текст:
[c],[l],[o],[s],[e],[u],[p],[ ],[o],[f],[ ],[b],[i],[n],[s],[ ],[o],[f],[ ],[f],[o],[o],[d],[ ],[t],[h],[a],[t],[ ],[i],[n],[c],[l],[u],[d],[e],[ ],[b],[r],[o],[c],[c],[o],[l],[i],[ ],[a],[n],[d],[ ],[b],[r],[e],[a],[d],[ ],[.]
[a],[ ],[m],[e],[a],[l],[ ],[i],[s],[ ],[p],[r],[e],[s],[e],[n],[t],[e],[d],[ ],[i],[n],[ ],[b],[r],[i],[g],[h],[t],[l],[y],[ ],[c],[o],[l],[o],[r],[e],[d],[ ],[p],[l],[a],[s],[t],[i],[c],[ ],[t],[r],[a],[y],[s],[ ],[.]
[t],[h],[e],[r],[e],[ ],[a],[r],[e],[ ],[c],[o],[n],[t],[a],[i],[n],[e],[r],[s],[ ],[f],[i],[l],[l],[e],[d],[ ],[w],[i],[t],[h],[ ],[d],[i],[f],[f],[e],[r],[e],[n],[t],[ ],[k],[i],[n],[d],[s],[ ],[o],[f],[ ],[f],[o],[o],[d],[s]
[c],[o],[

### Сохранение векторов

Если вы делаете это задание на `Colab`, сохраните папку с данными себе на диск или скачайте её, чтобы можно было переиспользовать в следующей части задания.

In [20]:
!mkdir data
np.save("./data/image_codes.npy", np.asarray(vectors))

with open('./data/captions_tokenized.json', 'w') as f_cap:
    json.dump(captions_tokenized, f_cap)

## Часть 2. Генерации описания изображения.

![img](https://docs.google.com/uc?export=download&id=1S084CyPJx-Y8qO14rq04m3QPH3auWdni)


### Загрузка и чтение данных

Загрузим векторы изображений, полученные в первой части задания. 



Если вы не выполнили ту часть задания и не претендуете на баллы за неё, <a href="https://drive.google.com/file/d/16Lg6FjUfzGpvHgzHRHS9VD2L3Rv_862P/view?usp=sharing">загрузите</a> изображения, которые мы векторизовали за вас при помощи `ResNet`. 
Поскольку этот файл большой, для его скачивания с Google.Drive применим технику описанную в <a href="https://medium.com/@acpanjan/download-google-drive-files-using-wget-3c2c025a8b99">этой статье</a>.

In [21]:
# скачаем векторы изображений через wget
!wget --load-cookies /tmp/cookies.txt\
  "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=16Lg6FjUfzGpvHgzHRHS9VD2L3Rv_862P' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=16Lg6FjUfzGpvHgzHRHS9VD2L3Rv_862P" \
  -O handout.tar.gz\
  && rm -rf /tmp/cookies.txt

# распакуем архив, содержащий папку data со всеми данными
!tar -xzf handout.tar.gz

# проверим содержимое
!ls -lha data

--2022-04-29 14:32:13--  https://docs.google.com/uc?export=download&confirm=t&id=16Lg6FjUfzGpvHgzHRHS9VD2L3Rv_862P
Resolving docs.google.com (docs.google.com)... 142.251.120.138, 142.251.120.101, 142.251.120.139, ...
Connecting to docs.google.com (docs.google.com)|142.251.120.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-10-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/uc4on6du80rb8s04i25qqplrkhsvamhd/1651242675000/14359032242157329066/*/16Lg6FjUfzGpvHgzHRHS9VD2L3Rv_862P?e=download [following]
--2022-04-29 14:32:13--  https://doc-10-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/uc4on6du80rb8s04i25qqplrkhsvamhd/1651242675000/14359032242157329066/*/16Lg6FjUfzGpvHgzHRHS9VD2L3Rv_862P?e=download
Resolving doc-10-4c-docs.googleusercontent.com (doc-10-4c-docs.googleusercontent.com)... 64.233.183.132, 2607:f8b0:4001:c0b::84
Connecting to doc-10-4c-docs.googleusercontent.com (doc-1

Прочтём данные из файлов.

In [54]:
%%time

img_codes = np.load("data/image_codes.npy")
captions = json.load(open('data/captions_tokenized.json'))

CPU times: user 222 ms, sys: 1.36 s, total: 1.58 s
Wall time: 1.79 s


### Предобработка данных

Убедимся, что векторы для всех изображений имеют размер 2048.

In [55]:
print("Размерность всех данных", img_codes.shape)
print(img_codes[0, :10])

Размерность всех данных (118287, 2048)
[0.3659946  0.2016555  0.9245725  0.57063824 0.547268   0.8275868
 0.3687277  0.12085301 0.0561931  0.49758485]


Каждое изображение имеет 5 вариантов текстового описания, причём все тексты токенизированы и приведены в нижний регистр.

In [56]:
print('\n'.join(captions[228]))

two people playing tennis in a neighborhood park .
two people on the tennis court playing tennis
two people at a public tennis court ; one serves the ball .
a tennis player taking a swing at a ball
two people playing tennis on a tennis court .


Разобъём текст на токены. Заведём отдельные токены для начала и конца текста.

In [57]:
START_TOKEN, END_TOKEN = '#BOS', '#EOS'
captions = [[[START_TOKEN] + caption.split(' ') + [END_TOKEN]\
            for caption in img_caption_list]\
            for img_caption_list in captions]

In [59]:
captions[:1]

[[['#BOS',
   'people',
   'shopping',
   'in',
   'an',
   'open',
   'market',
   'for',
   'vegetables',
   '.',
   '#EOS'],
  ['#BOS',
   'an',
   'open',
   'market',
   'full',
   'of',
   'people',
   'and',
   'piles',
   'of',
   'vegetables',
   '.',
   '#EOS'],
  ['#BOS',
   'people',
   'are',
   'shopping',
   'at',
   'an',
   'open',
   'air',
   'produce',
   'market',
   '.',
   '#EOS'],
  ['#BOS',
   'large',
   'piles',
   'of',
   'carrots',
   'and',
   'potatoes',
   'at',
   'a',
   'crowded',
   'outdoor',
   'market',
   '.',
   '#EOS'],
  ['#BOS',
   'people',
   'shop',
   'for',
   'vegetables',
   'like',
   'carrots',
   'and',
   'potatoes',
   'at',
   'an',
   'open',
   'air',
   'market',
   '.',
   '#EOS']]]

Вычислим количество уникальных слов, встречающихся в описании.

In [60]:
word_set = np.unique(captions)
print(len(word_set))

/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:270: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ar = np.asanyarray(ar)


567802


Заметим, что слов очень много. Для того, чтобы ограничить словарь, выберем наиболее частые слова.

In [61]:
all_words = []
for text in captions:
    for word in text[0]:
        all_words.append(word)

all_words[:10]

['#BOS',
 'people',
 'shopping',
 'in',
 'an',
 'open',
 'market',
 'for',
 'vegetables',
 '.']

In [62]:
word_counts = Counter(all_words)

# Посчитайте количество вхождений каждого уникального слова
word_counts.items()

dict_items([('#BOS', 118287), ('people', 8187), ('shopping', 77), ('in', 34441), ('an', 9025), ('open', 1266), ('market', 253), ('for', 1823), ('vegetables', 871), ('.', 92227), ('#EOS', 118287), ('a', 205720), ('person', 4769), ('skiing', 615), ('down', 5058), ('steep', 42), ('hill', 712), ('the', 34474), ('black', 3865), ('and', 25914), ('white', 8362), ('dog', 3672), ('stands', 872), ('near', 3678), ('holding', 6530), ('frisbee', 1652), ('skateboarder', 394), ('is', 16599), ('putting', 159), ('on', 43180), ('show', 116), ('using', 569), ('picnic', 89), ('table', 6556), ('as', 1276), ('his', 3311), ('stage', 38), ('vintage', 144), ('photo', 953), ('of', 45412), ('two', 8995), ('men', 2334), ('lama', 3), ('group', 5067), ('standing', 10387), ('around', 1752), ('each', 1808), ('other', 2687), ('ski', 660), ('equipment', 166), ('bathroom', 2828), ('with', 30352), ('toilet', 2216), (',', 7056), ('sink', 1696), ('bathtub', 186), ('painted', 258), ('yellow', 1788), ('large', 5309), ('motor

Сформируем словарь из наиболее частых токенов. Нам понадобятся 4 вспомогательных токена: 
* `#UNK` &mdash; слово, не встречающееся в словаре;
* `#BOS` &mdash; начало текста;
* `#EOS` &mdash; конец текста;
* `#PAD` &mdash; пустой токен.

In [63]:
vocab  = ['#UNK', '#BOS', '#EOS', '#PAD']
vocab += [k for k, v in word_counts.items() if v >= 5 if k not in vocab]
n_tokens = len(vocab)
print(n_tokens)
# assert 10000 <= n_tokens <= 10500

word_to_index = {w: i for i, w in enumerate(vocab)}

4742


In [64]:
eos_ix = word_to_index['#EOS']
unk_ix = word_to_index['#UNK']
pad_ix = word_to_index['#PAD']

def as_matrix(sequences, max_len=None):
    """ Конвертирование списка строк в целочисленный тензор """

    max_len = max_len or max(map(len,sequences))
    matrix = np.zeros((len(sequences), max_len), dtype='int32') + pad_ix

    for i,seq in enumerate(sequences):
        row_ix = [word_to_index.get(word, unk_ix) for word in seq[:max_len]]
        matrix[i, :len(row_ix)] = row_ix
    
    return matrix

Проверим работу функции на случайном описании.

In [65]:
as_matrix(captions[1337])

array([[   1,   13,  110,   61,    6,  202,   41,   13,  366,   68,   12,
           2,    3,    3,    3,    3,    3,    3,    3],
       [   1,   13,  110,  111,  112,   13,  366,   68,    2,    3,    3,
           3,    3,    3,    3,    3,    3,    3,    3],
       [   1,   13,  110,   29,  112,  972,   31,  310,   68,    2,    3,
           3,    3,    3,    3,    3,    3,    3,    3],
       [   1,   19,  244,  110,   29,  111,  112,  310,  659,   78,   12,
           2,    3,    3,    3,    3,    3,    3,    3],
       [   1,   13,  110,  111,  112,   13,  366,   54,   52,   19, 1253,
           0,   13,    0,   31,   19,  479,   12,    2]], dtype=int32)

### Архитектура декодера

В качестве модели будем использовать `CaptionNet`.

![img](https://raw.githubusercontent.com/yunjey/pytorch-tutorial/master/tutorials/03-advanced/image_captioning/png/model.png)


Эта архитектура &mdash; своеобразная модификация рекуррентной языковой модели. Основная особенность &mdash; начальные скрытые  состояния LSTM вычисляются по изображению при помощи свёрточной нейронной сети. Мы уже получили векторы изображений в первой части задания. Поэтому сейчас нам не придётся использовать свёрточные слои в явном виде.


In [82]:
class CaptionNet(nn.Module):
    def __init__(self, n_tokens=n_tokens, emb_size=128, lstm_units=256,
                 cnn_feature_size=2048):
        """ 
        Инициализация CaptionNet
        
        Параметры.
        1) n_tokens - размер словаря для текстовых описаний,
        2) emb_size - размер эмбеддингов для токенов из словаря,
        3) lstm_units - размер скрытого состояния LSTM,
        4) cnn_features_size - размер эмбеддинга изображений.
        """

        super(self.__class__, self).__init__()
        
        # слой, переводящий эмбеддинг картинки в вектор, 
        # который будет использован как начальное состояние h в LSTM
        self.cnn_to_h0 = nn.Linear(cnn_feature_size, lstm_units)
        # слой, переводящий эмбеддинг картинки в вектор, 
        # который будет использован как начальное состояние c в LSTM
        self.cnn_to_c0 = nn.Linear(cnn_feature_size, lstm_units)
        # эмбеддинги для токенов описаний
        self.emb = nn.Embedding(n_tokens, emb_size)
        # рекуррентный слой
        self.lstm = nn.LSTM(emb_size, lstm_units, batch_first=True)
        # слой для вычисления логитов
        self.logits = nn.Linear(lstm_units, n_tokens)
        
    def forward(self, image_vectors, captions_ix):
        """ Применение сети """

        initial_cell = self.cnn_to_c0(image_vectors)
        initial_hid = self.cnn_to_h0(image_vectors)
        
        # получим эмбеддинги описаний
        captions_emb = self.emb(captions_ix)
        # применяем lstm, в качестве начальных состояний берем initial_cell и initial_hid
        lstm_out, _ = self.lstm(captions_emb, (initial_cell.unsqueeze(0), initial_hid.unsqueeze(0)))
        # вычислим логиты
        logits = self.logits(lstm_out)
        
        return logits        

Инициализируем сеть.

In [83]:
network = CaptionNet(n_tokens)
checkpoint_path = Path('best_model.pt')

Сгенерируем случайные векторы для простого тестирования модели.

In [84]:
dummy_img_vec = torch.randn(len(captions[0]), 2048)
dummy_capt_ix = torch.tensor(as_matrix(captions[0]), dtype=torch.int64)

dummy_logits = network.forward(dummy_img_vec, dummy_capt_ix)

print('shape:', dummy_logits.shape)
assert dummy_logits.shape == (dummy_capt_ix.shape[0], dummy_capt_ix.shape[1], n_tokens)

shape: torch.Size([5, 16, 4742])


Реализуем вычисление функции потерь.

In [85]:
def compute_loss(network, image_vectors, captions_ix):
    """ Подсчёт функции потерь """
    
    captions_ix_inp = captions_ix[:, :-1].contiguous()
    captions_ix_next = captions_ix[:, 1:].contiguous()
    
    # получим логиты, применив сеть
    logits_for_next = network.forward(image_vectors, captions_ix_inp)    
    loss = F.cross_entropy(
        logits_for_next.view(-1, n_tokens), 
        captions_ix_next.view(-1), 
        ignore_index=pad_ix
    )
    
    return torch.stack([loss])

Посчитаем функцию потерь на простом примере.

In [86]:
dummy_loss = compute_loss(network, dummy_img_vec, dummy_capt_ix)
dummy_loss.backward()

Инициализируем оптимизатор для нейронной сети. 

In [87]:
optim = torch.optim.Adam(network.parameters(), lr=1e-4)

### Обучение сети


Разделим данные на обучение и валидацию. Отведём на валидацию 20% данных.

In [88]:
captions = np.array(captions)
train_img_codes, val_img_codes, train_captions, val_captions = train_test_split(
    img_codes, captions, test_size=0.2, random_state=42
)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.


Реализуем генератор батчей.

In [89]:
def generate_batch(img_codes, captions, batch_size, max_caption_len=None):
    '''
    Генератор батчей.
    Параметры.
    1) img_codes - векторы изображений,
    2) captions - описания изображений,
    3) batch_size - размер батча,
    4) max_caption_len - ограничение сверху на длину описания.
    '''

    # выбираем случайные изображения
    random_image_ix = np.random.randint(0, len(img_codes), size=batch_size)
    batch_images = img_codes[random_image_ix]
    
    # берём все описания для выбранных случайных изображений
    captions_for_batch_images = captions[random_image_ix]
    
    # берём для каждого изображения ровно одно описание
    batch_captions = list(map(choice, captions_for_batch_images))
    batch_captions_ix = as_matrix(batch_captions,max_len=max_caption_len)
    
    return torch.tensor(batch_images, dtype=torch.float32),\
           torch.tensor(batch_captions_ix, dtype=torch.int64)

Сгенерируем один случайный батч.

In [90]:
generate_batch(img_codes, captions, 3)

(tensor([[0.4021, 0.1639, 0.9798,  ..., 0.0059, 0.7763, 0.0145],
         [0.0391, 0.2453, 0.1750,  ..., 0.0016, 0.0156, 0.0000],
         [0.4699, 0.1062, 0.1570,  ..., 0.0641, 0.8460, 0.1538]]),
 tensor([[   1,    4,   45,  344,   33,  532,  534,   31,    0,    2,    3,    3],
         [   1,   13,  629,   29,   26,   13,  456,   21,   13,  203,  437,    2],
         [   1,  339, 2354,  847,  137,  892,   19, 1631,  112,   19,  526,    2]]))

### Цикл обучения
 

Подберите размер батча и количество эпох в зависимости от имеющихся вычислительных ресурсов.

In [91]:
batch_size = 64
n_epochs = 5
n_batches_per_epoch = len(train_img_codes) // batch_size 
n_validation_batches = len(train_img_codes) // batch_size

Обучим модель.

In [92]:
best_val_loss = 1000.0

for epoch in range(n_epochs):
    train_loss = 0
    network.train(True)
    
    for _ in tqdm(range(n_batches_per_epoch)):
        # считаем значение функции потерь
        loss_t = compute_loss(
            network, 
            *generate_batch(train_img_codes, train_captions, batch_size)
        )

        optim.zero_grad()
        # делаем обратное распространение ошибки
        loss_t.backward()
        # делаем шаг оптимизатора
        optim.step()
        train_loss += loss_t.item()

    train_loss /= n_batches_per_epoch
    
    val_loss = 0
    network.train(False)
    for _ in range(n_validation_batches):
        loss_t = compute_loss(
            network, 
            *generate_batch(val_img_codes, val_captions, batch_size)
        )
        val_loss += loss_t.item()
    val_loss /= n_validation_batches
    
    print('\nЭпоха номер: {}, train loss: {}, val loss: {}'.format(epoch, train_loss, val_loss))

    if val_loss < best_val_loss:
        torch.save(network.state_dict(), checkpoint_path)
        print('Сохранено')

  0%|          | 0/1478 [00:00<?, ?it/s]


Эпоха номер: 0, train loss: 4.608190547790837, val loss: 4.008132019610786
Сохранено


  0%|          | 0/1478 [00:00<?, ?it/s]


Эпоха номер: 1, train loss: 3.737450279466838, val loss: 3.5233361966232484
Сохранено


  0%|          | 0/1478 [00:00<?, ?it/s]


Эпоха номер: 2, train loss: 3.3710989345555697, val loss: 3.2493855130527276
Сохранено


  0%|          | 0/1478 [00:00<?, ?it/s]


Эпоха номер: 3, train loss: 3.1507379678008682, val loss: 3.0868176741270963
Сохранено


  0%|          | 0/1478 [00:00<?, ?it/s]


Эпоха номер: 4, train loss: 3.014389465561739, val loss: 2.9843216937675527
Сохранено


### Применение обученной модели 

На этом этапе мы попытаемся применить обученную сеть к своим изображениям. Для начала загрузим сеть, с использованием которой была выполнена векторизация изображений.



In [ ]:
vectorizer_model = <...>

### Генерация описаний

Реализуем функцию для генерации текстовых описаний по изображению.

In [ ]:
def generate_caption(
    image, 
    caption_prefix=("#START"), 
    temp=1, 
    sample=True, 
    max_tokens=100
):
    """
    Генерация описаний изображений.

    Параметры.
    1) image - изображение в формате RGB,
    2) caption_prefix - начало сгенерированного текста,
    3) temp - температура,
    4) sample - при установке в True сэмплирует следующий токен на каждом этапе,
    иначе - применяет жадный алгоритм,
    5) max_tokens - максимальное количество токенов в итоговом описании.
    """
  
    assert isinstance(image, np.ndarray) and np.max(image) <= 1\
           and np.min(image) >= 0 and image.shape[-1] == 3
    
    # отключим вычисление градиентов
    with torch.no_grad():
        # преобразуем изображение в тензор
        image = torch.tensor(image.transpose([2, 0, 1]), dtype=torch.float32)
        vectors_neck = vectorizer_model(image[None])
        caption_prefix = list(caption_prefix)

        for _ in range(max_tokens):
            prefix_ix = as_matrix([caption_prefix])
            prefix_ix = torch.tensor(prefix_ix, dtype=torch.int64)
            # вычисляем логиты и вероятности всех токенов словаря
            next_word_logits = <...>
            next_word_probs = <...>

            assert len(next_word_probs.shape) == 1, 'вектор вероятностей должен быть одномерным'
            
            next_word_probs = <...> 
            # генерируем следующий токен описания
            if sample:
                next_word = <...>
            else:
                next_word = <...>
                
            # добавляем сгенерированный токен в ответ
            caption_prefix.append(next_word)
            
            # если увидели токен конца текста, завершаем процесс
            if next_word == "#EOS":
                break
            
    return caption_prefix

А теперь возьмите 5 любых изображений из интернета, желательно, сильно отличающихся друг от друга, и сгенерируйте для них описания с помощью обученной модели.

In [ ]:
<...>